In [9]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
from nanrms import nansumsq, nanrms

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

# Collect Data Files

In [10]:
datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
exp = "_Bandlock_Udp_B1_B3_B7_B8_RM500Q"
devices = [
    'qc00',
    # 'qc01',
    'qc02',
    'qc03',
]
schemes = [
    'B1',
    # 'B3',
    'B7',
    'B8',
]
dates = {
    "2023-03-15": [*['#{:02d}'.format(i+1) for i in range(1, 4)], '#06'],
    "2023-03-16": ['#{:02d}'.format(i+1) for i in range(16)],
}

In [11]:
def set_data(df):
    df['seq'] = df['seq'].astype('Int32')
    df['rpkg'] = df['rpkg'].astype('Int8')
    df['frame_id'] = df['frame_id'].astype('Int32')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['xmit_time'] = pd.to_datetime(df['xmit_time'])
    df['arr_time'] = pd.to_datetime(df['arr_time'])
    df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
    df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
    df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
    df['lost'] = df['lost'].astype('boolean')
    df['excl'] = df['excl'].astype('boolean')
    df['latency'] = df['latency'].astype('float32')
    return df

## Uplink

In [12]:
i = -1
dfs_ul = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        if i in [16,17,18,19]:
            dfs_ul.append([pd.DataFrame()]*len(devices))
            continue
        dfs_ul.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            # print(data, os.path.isfile(data))
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_ul[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ul))

20


## Downlink

In [13]:
i = -1
dfs_dl = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        if i in [0,1,2,3]:
            dfs_dl.append([pd.DataFrame()]*len(devices))
            continue
        dfs_dl.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            # print(data, os.path.isfile(data))
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_dl[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_dl))

20


In [14]:
df = dfs_ul[0][0]

plr = round(df.lost.mean() * 100, 4)
elr = round(df[~df.lost].excl.mean() * 100, 4)
plelr = round(df.excl.mean() * 100, 4)
avg_l = round(df[~df.lost].latency.mean(), 6)
min_l = round(df[~df.lost].latency.min(), 6)
max_l = round(df[~df.lost].latency.max(), 6)
mdn_l = round(df[~df.lost].latency.median(), 6)
var_l = round(df[~df.lost].latency.var(), 6)
std_l = round(df[~df.lost].latency.std(), 6)
jitter = round(df.loc[~df.lost, 'latency'].diff().abs().mean(), 6)

print('packet loss rate (%):      ', plr)
print('excessive latency rate (%):', elr)
print('plr + elr (%):             ', plelr)
print('average latency:           ', avg_l)
print('min latency:               ', min_l)
print('max latency:               ', max_l)
print('median latency:            ', mdn_l)
print('latency variance:          ', var_l)
print('latency standard deviation:', std_l)
print('latency jitter:            ', jitter)

packet loss rate (%):       0.1094
excessive latency rate (%): 0.0007
plr + elr (%):              0.1101
average latency:            0.015278
min latency:                0.004967
max latency:                0.100752
median latency:             0.014498
latency variance:           1.5e-05
latency standard deviation: 0.003862
latency jitter:             0.001796


# Statistics

In [15]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            lost_ul = dfs_ul[i][j].lost.sum() if not dfs_ul[i][j].empty else np.nan
            excl_ul = dfs_ul[i][j][~dfs_ul[i][j].lost].excl.sum() if not dfs_ul[i][j].empty else np.nan
            sent_ul = dfs_ul[i][j].seq.count() if not dfs_ul[i][j].empty else np.nan
            lost_dl = dfs_dl[i][j].lost.sum() if not dfs_dl[i][j].empty else np.nan
            excl_dl = dfs_dl[i][j][~dfs_dl[i][j].lost].excl.sum() if not dfs_dl[i][j].empty else np.nan
            sent_dl = dfs_dl[i][j].seq.count() if not dfs_dl[i][j].empty else np.nan
            # print(lost_ul, excl_ul, sent_ul, lost_dl, excl_dl, sent_dl, sep='\t', end='\t')
            row = [*row, lost_ul, excl_ul, sent_ul, lost_dl, excl_dl, sent_dl]
        # print()
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_lost_{schm}', f'ul_excl_{schm}', f'ul_total_{schm}', f'dl_lost_{schm}', f'dl_excl_{schm}', f'dl_total_{schm}']
average = [round(s, 1) for s in np.nanmean(table, axis=0)]
with open("stats_lost_excl.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

In [16]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            plr_ul = round(dfs_ul[i][j].lost.mean() * 100, 4) if not dfs_ul[i][j].empty else np.nan
            elr_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].excl.mean() * 100, 4) if not dfs_ul[i][j].empty else np.nan
            plr_dl = round(dfs_dl[i][j].lost.mean() * 100, 4) if not dfs_dl[i][j].empty else np.nan
            elr_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].excl.mean() * 100, 4) if not dfs_dl[i][j].empty else np.nan
            # print(plr_ul, elr_ul, plr_dl, elr_dl, sep='\t', end='\t')
            row = [*row, plr_ul, elr_ul, plr_dl, elr_dl]
        # print()
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_PLR_{schm}', f'ul_ELR_{schm}', f'dl_PLR_{schm}', f'dl_ELR_{schm}']
average = [round(s, 4) for s in np.nanmean(table, axis=0)]
with open("stats_plr_elr.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

In [229]:
i = -1
table = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        row = []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            avg_l_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].latency.mean(), 6) if not dfs_ul[i][j].empty else np.nan
            std_l_ul = round(dfs_ul[i][j][~dfs_ul[i][j].lost].latency.std(), 6) if not dfs_ul[i][j].empty else np.nan
            jitter_ul = round(dfs_ul[i][j].loc[~dfs_ul[i][j].lost, 'latency'].diff().abs().mean(), 6) if not dfs_ul[i][j].empty else np.nan
            avg_l_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].latency.mean(), 6) if not dfs_dl[i][j].empty else np.nan
            std_l_dl = round(dfs_dl[i][j][~dfs_dl[i][j].lost].latency.std(), 6) if not dfs_dl[i][j].empty else np.nan
            jitter_dl = round(dfs_dl[i][j].loc[~dfs_dl[i][j].lost, 'latency'].diff().abs().mean(), 6) if not dfs_dl[i][j].empty else np.nan
            # print(avg_l_ul, std_l_ul, jitter_ul, avg_l_dl, std_l_dl, jitter_dl, sep='\t', end='\t')
            row = [*row, avg_l_ul, std_l_ul, jitter_ul, avg_l_dl, std_l_dl, jitter_dl]
        # print()    
        table.append(row)

header = []
for dev, schm in zip(devices, schemes):
    header = [*header, f'ul_avg_L_{schm}', f'ul_std_L_{schm}', f'ul_jitter_{schm}', f'dl_avg_L_{schm}', f'dl_std_L_{schm}', f'dl_jitter_{schm}']
average = [round(s[1], 6) if i%3 == 1 else round(s[0], 6) for i, s in enumerate(zip(np.nanmean(table, axis=0), nanrms(table, axis=0)))]
with open("stats_latency.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(table)
    writer.writerow(average)

# Handover Info

In [7]:
from handover import mi_parse_ho, get_ho_interval, label_ho_info

i = -1
dfs_ho = []
for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            path = os.path.join(datadir, date, exp, dev, trace, 'data')
            data = os.path.join(path, [s for s in os.listdir(path) if s.startswith('diag_log_') and s.endswith('_rrc.csv')][0])
            print(data, os.path.isfile(data))
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            dfs_ho[i].append(df.copy())
print(len(dfs_ho))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#02/data/diag_log_qc00_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc02/#02/data/diag_log_qc02_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc03/#02/data/diag_log_qc03_2023-03-15_15-15-41_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#03/data/diag_log_qc00_2023-03-15_15-57-16_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc02/#03/data/diag_log_qc02_2023-03-15_15-57-16_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc03/#03/data/diag_log_qc03_2023-03-15_15-57-16_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-15/_Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#04/data/diag_log_qc00_2023